In [ ]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pyperclip

import re
import time
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format
import seaborn as sns

import koreanize_matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from wordcloud import WordCloud

font_path = '/Users/jieunlee/Documents/3_TECH!T_202305/TECH!T_dataton/nanum-barun-gothic/NanumBarunGothic.ttf'  # 각자 경로에 맞게
fontprop = fm.FontProperties(fname=font_path, size=18)

# **전처리**

In [ ]:
df = pd.read_csv('crawling_total_한국정보추가_908_230917.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  908 non-null    int64  
 1   제목          908 non-null    object 
 2   작가          908 non-null    object 
 3   장르          908 non-null    object 
 4   좋아요수        908 non-null    object 
 5   조회수         908 non-null    object 
 6   구독수         908 non-null    object 
 7   별점          908 non-null    float64
 8   회차          908 non-null    int64  
 9   Url         598 non-null    object 
 10  국가          298 non-null    object 
 11  기타url       32 non-null     object 
 12  한국제목        598 non-null    object 
 13  한국작가명       598 non-null    object 
 14  회차(한국)      598 non-null    float64
 15  별점(한국)      598 non-null    float64
 16  관심수(한국)     598 non-null    object 
 17  장르(한국)      598 non-null    object 
 18  연재시작일       598 non-null    object 
 19  마지막연재일      598 non-null    o

In [ ]:
df.head()

,Unnamed: 0,제목,작가,장르,좋아요수,조회수,구독수,별점,회차,Url,국가,기타url,한국제목,한국작가명,회차(한국),별점(한국),관심수(한국),장르(한국),연재시작일,마지막연재일,완결여부,작가전작개수,OSMU여부
0,0,ปฏิบัติการลับบุกโรงเรียนไฮโซ,AJ,แอกชัน,"22,258","672,894","89,029",9.88000,4,https://comic.naver.com/webtoon/list?titleId=8...,NaN,NaN,잠입! 재벌고,글/그림 AJ,41.00000,9.76000,192515,"액션,재벌,학원로맨스",23.02.26,23.02.26,False,1.00000,False
1,1,ห้องนอนลับของเจ้าหญิงต้องสาป,Henie / HYERIM SUNG,โรแมนซ์แฟนตาซี,"568,221",18.9M,"313,278",9.73000,54,https://comic.naver.com/webtoon/list?titleId=7...,NaN,NaN,버림받은 왕녀의 은밀한 침실,글/그림 혜니 원작 성혜림,68.00000,9.94000,290951,"로맨스,고자극로맨스,궁중로맨스,까칠남,소설원작,성인웹툰",22.07.03,23.09.10,False,6.00000,False
2,2,ช่วยเปลี่ยนฉันที,ลีจีโฮ / โฮตี,ดราม่า,1.7M,46.9M,"358,044",9.62000,193,https://comic.naver.com/webtoon/list?titleId=7...,NaN,NaN,나를 바꿔줘,글 이지호 그림 호띠,215.00000,9.69000,259728,드라마,19.09.06,23.09.08,False,1.00000,False
3,3,นักล่าเงิน,EUN RYU / DO NEUK,แอกชัน,"1,929","96,493","17,986",9.80000,1,https://comic.naver.com/webtoon/list?titleId=8...,NaN,NaN,돈내놔,글 은류 그림 도늑,16.00000,8.20000,26261,"액션,서바이벌,사이다,게임판타지,두뇌싸움,먼치킨,머니게임",23.06.28,23.09.06,False,1.00000,False
4,4,ถูกใจนายติดอ่างที่รัก,แผ่นทองสีดำ,LGBTQ+ / Y,1.4M,30.8M,"360,472",9.84000,95,NaN,태국,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

### **1. 중복확인**

In [ ]:
df[df.duplicated(['제목', '작가'], keep=False)]

,제목,작가,장르,좋아요수,조회수,구독수,별점,회차,Url,국가,기타url,한국제목,한국작가명,회차(한국),별점(한국),관심수(한국),장르(한국),연재시작일,마지막연재일,완결여부,작가전작개수,OSMU여부
11,แฮปปี้ควอกก้า,WEBTOON,ทิปตูน,"98,628",8.3M,"84,991",9.23000,127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
28,เว็บตูนอีเวนต์,WEBTOON,ทิปตูน,"73,905",10.7M,"127,549",9.41000,156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
32,กลั้นน้ำตา,Pixel Ghost,ชีวิตประจำวัน,3.3M,97.3M,"410,015",9.62000,1004,NaN,태국,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
57,แฮปปี้ควอกก้า,WEBTOON,ทิปตูน,"98,628",8.3M,"84,993",9.23000,127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
78,เว็บตูนอีเวนต์,WEBTOON,ทิปตูน,"73,905",10.7M,"127,549",9.41000,156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
99,Daily JoJo,ลีดงกอน,ชีวิตประจำวัน,"169,610",2.5M,"29,229",9.78000,100,https://comic.naver.com/webtoon/list?titleId=7...,NaN,NaN,조조코믹스,글/그림 이동건,101.00000,9.93000,395679,"로맨스,비밀연애,현실로맨스,오피스로맨스,사내연애",21.07.16,22.10.11,False,13.00000,False
119,แฮปปี้ควอกก้า,WEBTOON,ทิปตูน,"98,628",8.3M,"84,993",9.23000,127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
133,เว็บตูนอีเวนต์,WEBTOON,ทิปตูน,"73,905",10.7M,"127,549",9.41000,156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
170,แฮปปี้ควอกก้า,WEBTOON,ทิปตูน,"98,628",8.3M,"84,995",9.23000,127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
187,เว็บตูนอีเวนต์,WEBTOON,ทิปตูน,"73,905",10.7M,"127,549",9.41000,156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
# 작가 == 'WEBTOON', 'Various Artist' 전부 삭제하기
df = df[(df['작가'] != 'WEBTOON') & (df['작가'] != 'Various Artists')]

In [ ]:
# 'กลั้นน้ำตา', 'Daily JoJo' -> 일주일에 2번 이상 연재하는 경우 -> 첫번째 행만 남기고 삭제하기
df.drop_duplicates(subset=['제목', '작가'], keep='first', inplace=True)

In [ ]:
# 확인
df[df.duplicated(['제목', '작가'])]

,제목,작가,장르,좋아요수,조회수,구독수,별점,회차,Url,국가,기타url,한국제목,한국작가명,회차(한국),별점(한국),관심수(한국),장르(한국),연재시작일,마지막연재일,완결여부,작가전작개수,OSMU여부


In [ ]:
df.reset_index(inplace=True, drop=True)

### **2. 결측치 채우기**

In [ ]:
df['국가'].fillna('한국', inplace=True)

### **3. 새 칼럼 추가**

**장르(태국)**

In [ ]:
df['장르'].value_counts()

장르
โรแมนซ์           222
โรแมนซ์แฟนตาซี    130
ดราม่า            105
แอกชัน            101
แฟนตาซี            86
ระทึกขวัญ          58
LGBTQ+ / Y         55
ตลก                40
นิยาย              27
อบอุ่นหัวใจ        21
ชีวิตประจำวัน      17
สยองขวัญ           14
ทิปตูน              1
Name: count, dtype: int64

In [ ]:
# 장르(태국) 칼럼 추가
df.loc[df['장르'] == 'โรแมนซ์', '장르(태국)'] = '로맨스'
df.loc[df['장르'] == 'โรแมนซ์แฟนตาซี', '장르(태국)'] = '로맨스판타지'
df.loc[df['장르'] == 'ดราม่า', '장르(태국)'] = '드라마'
df.loc[df['장르'] == 'แอกชัน', '장르(태국)'] = '액션'
df.loc[df['장르'] == 'แฟนตาซี', '장르(태국)'] = '판타지'
df.loc[df['장르'] == 'ระทึกขวัญ', '장르(태국)'] = '스릴러'
df.loc[df['장르'] == 'LGBTQ+ / Y', '장르(태국)'] = 'LGBTQ+ / Y'
df.loc[df['장르'] == 'ตลก', '장르(태국)'] = '개그'
df.loc[df['장르'] == 'นิยาย', '장르(태국)'] = '소설'
df.loc[df['장르'] == 'อบอุ่นหัวใจ', '장르(태국)'] = '감성'
df.loc[df['장르'] == 'ชีวิตประจำวัน', '장르(태국)'] = '일상'
df.loc[df['장르'] == 'ทิปตูน', '장르(태국)'] = '팁툰'
df.loc[df['장르'] == 'สยองขวัญ', '장르(태국)'] = '공포'

In [ ]:
df['장르(태국)'].value_counts()

장르(태국)
로맨스           222
로맨스판타지        130
드라마           105
액션            101
판타지            86
스릴러            58
LGBTQ+ / Y     55
개그             40
소설             27
감성             21
일상             17
공포             14
팁툰              1
Name: count, dtype: int64

**장르(한국) 및 해시태그(한국)**

In [ ]:
df['해시태그(한국)'] = df['장르(한국)']

In [ ]:
df.loc[df['장르(한국)'].notnull(), '장르(한국)'] = df.loc[df['장르(한국)'].notnull(), '장르(한국)'].apply(lambda x: x.split(',')[0])

**웹툰/시리즈구분**

In [ ]:
df.loc[df['Url'].notna() & df['Url'].str.startswith('https://comic.naver.com/'), '웹툰/시리즈구분'] = '웹툰'
df.loc[df['Url'].notna() & df['Url'].str.startswith('https://series.naver.com/'), '웹툰/시리즈구분'] = '시리즈'

**국가 및 국가설명**

In [ ]:
df['국가설명'] = df['국가']

In [ ]:
df['국가'] = [state[0].split('(')[0] for state in df['국가'].str.split('/')]

In [ ]:
df['국가'].unique()

array(['한국', '태국', '중국', '일본', '인도네시아', '미국', '대만'], dtype=object)

In [ ]:
df['국가설명'].unique()

array(['한국', '태국', '중국', '일본', '인도네시아', '미국(UnOrdinary)',
       '한국(슈퍼스트링-이세계견문록)', '한국/소설', '한국/리디북스', '한국/카카오페이지', '한국/버프툰',
       '한국/카카오웹툰', '미국(Zatanna & the Ripper)', '미국', '대만',
       '한국/1부, 2부로 나뉘어짐 https://comic.naver.com/webtoon/list?titleId=687921',
       '한국/리디북스 카카오페이지', '한국/탑툰', '한국/만화경', '한국/피너툰', '한국/레진코믹스',
       '한국/표절연재중단', '한국/도전/버프툰', '한국/베스트도전/리디북스/카카오페이지/버프툰', '한국/뷰티'],
      dtype=object)

**태국진출여부**

In [ ]:
df.loc[df['국가'] != '태국', '태국진출여부'] = True 
df.loc[df['국가'] == '태국', '태국진출여부'] = False

### **4. 데이터 타입 변환**
- 좋아요수 / 조회수 / 구독수 / 회차 / 회차(한국) / 관심수(한국) -> int
- 별점 / 별점(한국) -> float
- 연재시작일 / 마지막연재일 -> 날짜형식통일 -> datetime
- 완결여부 / OSMU여부 / 태국진출여부 -> bool

In [ ]:
df.loc[df['좋아요수'] == 'ถูกใจ!', '좋아요수'] = 0
df.loc[df['좋아요수'].str.contains('M', na=False), '좋아요수'] = df.loc[df['좋아요수'].str.contains('M', na=False)]['좋아요수'].str.replace('M', '').astype('float') * 1000000
df.loc[df['좋아요수'].str.contains(',', na=False), '좋아요수'] = df.loc[df['좋아요수'].str.contains(',', na=False)]['좋아요수'].str.replace(',', '').astype('float')
df['좋아요수'] = df['좋아요수'].astype('int')

In [ ]:
df.loc[df['조회수'].str.contains('M', na=False), '조회수'] = df.loc[df['조회수'].str.contains('M', na=False)]['조회수'].str.replace('M', '').astype('float') * 1000000
df.loc[df['조회수'].str.contains(',', na=False), '조회수'] = df.loc[df['조회수'].str.contains(',', na=False)]['조회수'].str.replace(',', '').astype('float')
df['조회수'] = df['조회수'].astype('int')

In [ ]:
df.loc[df['구독수'].str.contains('M', na=False), '구독수'] = df.loc[df['구독수'].str.contains('M', na=False)]['구독수'].str.replace('M', '').astype('float') * 1000000
df.loc[df['구독수'].str.contains(',', na=False), '구독수'] = df.loc[df['구독수'].str.contains(',', na=False)]['구독수'].str.replace(',', '').astype('float')
df['구독수'] = df['구독수'].astype('int')

In [ ]:
df.loc[df['관심수(한국)'] == '좋아요', '관심수(한국)'] = 0
df['관심수(한국)'] = df['관심수(한국)'].fillna(-1).astype('int').replace({-1: 0})

In [ ]:
df['회차(한국)'] = df['회차(한국)'].fillna(-1).astype('int').replace({-1: 0})

In [ ]:
df['작가전작개수'] = df['작가전작개수'].fillna(-1).astype('int').replace({-1: 0})

In [ ]:
df['연재시작일'].fillna('.', inplace=True)

for i, date in enumerate(df['연재시작일']):
    if date == '.':
        date = date
    elif date.endswith('.'):
        date = date[:-1]
    df.loc[i, '연재시작일'] = date

In [ ]:
df['연재시작일'] = df['연재시작일'].str.split('.')

for i, date in enumerate(df['연재시작일']):
    if date == '.':
        date = '.'
    elif len(date[0]) == 2:
        date[0] = '20' + date[0]
        date = '.'.join(date)
    else:
        date = '.'.join(date)
    df.loc[i, '연재시작일'] = date

df.loc[df['연재시작일'] == '.', '연재시작일'] = np.NaN

df['연재시작일'] = pd.to_datetime(df['연재시작일'], format='%Y.%m.%d')

In [ ]:
df['마지막연재일'].fillna('.', inplace=True)

for i, date in enumerate(df['마지막연재일']):
    if date == '.':
        date = date
    elif date.endswith('.'):
        date = date[:-1]
    df.loc[i, '마지막연재일'] = date

In [ ]:
df['마지막연재일'] = df['마지막연재일'].str.split('.')

for i, date in enumerate(df['마지막연재일']):
    if date == '.':
        date = '.'
    elif len(date[0]) == 2:
        date[0] = '20' + date[0]
        date = '.'.join(date)
    else:
        date = '.'.join(date)
    df.loc[i, '마지막연재일'] = date

df.loc[df['마지막연재일'] == '.', '마지막연재일'] = np.NaN

df['마지막연재일'] = pd.to_datetime(df['마지막연재일'], format='%Y.%m.%d')

In [ ]:
df.describe()

,좋아요수,조회수,구독수,별점,회차,회차(한국),별점(한국),관심수(한국),연재시작일,마지막연재일,작가전작개수
count,877.00000,877.00000,877.00000,877.00000,877.00000,877.00000,593.00000,877.00000,593,593,877.00000
mean,438377.60889,14371411.19954,178354.66135,9.51016,39.59863,72.13683,9.60589,148630.06727,2020-02-01 10:02:13.558178816,2021-09-28 12:32:46.947723520,2.56899
min,0.00000,488.00000,32.00000,4.16000,1.00000,0.00000,0.00000,0.00000,2005-12-12 00:00:00,2008-12-24 00:00:00,0.00000
25%,56836.00000,1500000.00000,48520.00000,9.44000,4.00000,0.00000,9.59000,0.00000,2018-12-30 00:00:00,2020-10-24 00:00:00,0.00000
50%,161937.00000,4200000.00000,99403.00000,9.67000,9.00000,53.00000,9.86000,43015.00000,2020-12-26 00:00:00,2022-07-28 00:00:00,1.00000
75%,504545.00000,13600000.00000,223250.00000,9.80000,52.00000,103.00000,9.93000,195496.00000,2022-01-03 00:00:00,2023-09-06 00:00:00,3.00000
max,11700000.00000,556500000.00000,2500000.00000,9.97000,1004.00000,1238.00000,10.00000,2542741.00000,2023-09-12 00:00:00,2023-09-14 00:00:00,21.00000
std,781236.43032,32842112.65526,236108.20870,0.56070,77.29819,96.28972,0.72705,256828.06307,NaN,NaN,3.62844


### **5. 불필요한 칼럼 drop**

In [ ]:
df.drop(['Url', '기타url', '장르', '국가설명'], axis=1, inplace=True)

### **6. 칼럼 순서 변경**

In [ ]:
df.columns

Index(['제목', '작가', '좋아요수', '조회수', '구독수', '별점', '회차', '국가', '한국제목', '한국작가명',
       '회차(한국)', '별점(한국)', '관심수(한국)', '장르(한국)', '연재시작일', '마지막연재일', '완결여부',
       '작가전작개수', 'OSMU여부', '장르(태국)', '해시태그(한국)', '웹툰/시리즈구분', '태국진출여부'],
      dtype='object')

In [ ]:
df = df[['제목', '작가', '장르(태국)', '좋아요수', '조회수', '구독수', '별점', '회차', '국가', 
         '한국제목', '한국작가명', '회차(한국)', '별점(한국)', '관심수(한국)', '장르(한국)', '해시태그(한국)', 
         '연재시작일', '마지막연재일', '완결여부', '웹툰/시리즈구분', '작가전작개수', 'OSMU여부', '태국진출여부']]

In [ ]:
df.columns

Index(['제목', '작가', '장르(태국)', '좋아요수', '조회수', '구독수', '별점', '회차', '국가', '한국제목',
       '한국작가명', '회차(한국)', '별점(한국)', '관심수(한국)', '장르(한국)', '해시태그(한국)', '연재시작일',
       '마지막연재일', '완결여부', '웹툰/시리즈구분', '작가전작개수', 'OSMU여부', '태국진출여부'],
      dtype='object')

In [ ]:
df.to_csv('crawling_total_전처리완료_877_230918.csv', index=False)